In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pycountry
!pip install pmdarima

In [3]:
import os
import pandas as pd
import numpy as np
from collections import Counter
import seaborn as sns
import plotly.express as px
from prophet import Prophet
from statsmodels.tsa.seasonal import seasonal_decompose
from prophet.plot import add_changepoints_to_plot
from prophet.plot import plot
import matplotlib.pyplot as plt
import pycountry
import random
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [ ]:
list(pycountry.countries)[0]
Ecuador=pycountry.countries.get(name='Ecuador')
Ecuador

In [5]:
# import pycountry
# from googletrans import Translator

# # Initialize the translator
# translator = Translator()

# # Get subdivisions for Ecuador (or any country)
# country_code = "EC"  # ISO code for Ecuador
# subdivisions = pycountry.subdivisions.get(country_code=country_code)

# # Translate names to English
# all_subdivs_english = []
# for subdivision in subdivisions:
#     # print(subdivision.name)
#     # translated_name = translator.translate(subdivision.name, src='auto', dest='en').text
#     all_subdivs_english.append(subdivision.name)
#     # print(f"Original: {subdivision.name}, Translated: {translated_name}")

# # Print the list of translated names
# print(all_subdivs_english)


In [6]:
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        print(filename[:-4])
        file_name =filename[:-4]
        file_name=pd.read_csv(os.path.join(dirname, filename))
        print(file_name.head())
        print('*'*100)
        print(file_name.info())
        print('*'*100)
        print(file_name.describe())
        print('++'*100)

In [7]:
data='/kaggle/input/store-sales-time-series-forecasting'

In [8]:
my_test=pd.read_csv(f'{data}/test.csv',parse_dates=['date'])
oil=pd.read_csv(f'{data}/oil.csv',parse_dates=['date'])
train=pd.read_csv(f'{data}/train.csv',parse_dates=['date']) # main df # main df
stores=pd.read_csv(f'{data}/stores.csv') # can be merged with train - on store no
ts=pd.read_csv(f'{data}/transactions.csv',parse_dates=['date']) # can be merged with train - on store no and date
holiday=pd.read_csv(f'{data}/holidays_events.csv',parse_dates=['date'])

In [9]:
stores.rename(columns={'type':'store_type'},inplace=True)

In [ ]:
holiday.head()

In [11]:
holiday['date']=pd.to_datetime(holiday['date'])

In [ ]:
for col in holiday.columns:
    if holiday[col].dtype=='object': 
        print(col)
        print(holiday[col].unique())
        print('*'*10)

In [13]:
good_words=['Fundacion','Mundial','Cantonizacion','Independencia','Batalla','Navidad','futbol','Provincializacion','Carnaval','Friday','Dia','Monday','Viernes']
bad_words=['Terremoto']

In [14]:
# Create a regular expression pattern from the list of good words
pattern = '|'.join([f'\\b{word.lower()}\\b' for word in good_words])  # This ensures we match whole words
bad_pattern='|'.join([f'\\b{word.lower()}\\b' for word in bad_words])
# Apply the logic to classify holidays

holiday['category'] = np.where(holiday['description'].str.lower().str.contains(pattern, na=False), 'good', 'bad')


In [15]:
def classify_holiday(description):
    # Check for good words
    for word in good_words:
        if word.lower() in description.lower():
            return word  # Return 'good' if any word in good_words is found
    # Check for bad words
    for word in bad_words:
        if word.lower() in description.lower():
            return word  # Return 'bad' if any word in bad_words is found
    return 'neutral'  # Default to neutral if neither is found

# Apply the function to the DataFrame
holiday['category_reg'] = holiday['description'].apply(classify_holiday)

In [ ]:
pattern

In [ ]:
holiday[holiday['description'].str.contains('futbol')]

In [ ]:
holiday[holiday['category'].str.contains('bad')].head()

In [ ]:
holiday[holiday['type']=='Event'].head()

In [ ]:
Counter(holiday['description'].to_list())

In [ ]:
holiday['locale'].unique()

In [ ]:
holiday[holiday['locale']=='Regional']['locale_name'].unique()

In [ ]:
holiday[holiday['locale']=='Local']['locale_name'].unique()

In [ ]:
train['family'].unique()

In [ ]:
holiday[holiday['description'].str.contains("Navidad")]

holiday[holiday['type'].str.contains("Work")]

In [26]:
holiday_national=holiday[holiday['locale'].str.contains("National")]
holiday_Regional=holiday[holiday['locale'].str.contains("Regional")]
holiday_Local=holiday[holiday['locale'].str.contains("Local")]


In [27]:
df=train.merge(stores,on='store_nbr',how='left')


In [ ]:
df['state'].unique()

In [ ]:
df

In [ ]:
df['city'].unique()
holiday_Local['locale_name'].unique()

In [31]:
# all_subdivs_english

In [32]:
# result = pd.Series(df['state'].unique()).isin(all_subdivs_english).all()
# result

In [33]:
# set(df['state'].unique())-set(all_subdivs_english)

In [34]:
df=df.merge(ts,on=['date','store_nbr'],how='left')
# df['date']=pd.to_datetime(df['date'])
# df=df.merge(holiday[holiday['type'].str.contains("Work ")],on='date',how='right')

In [ ]:
df['family'].unique()

In [36]:
def plot_graphs(data):
    sns.lineplot(data=data,x='date',y='sales')
    

In [37]:
# df.groupby(by=['store_nbr','family']).apply(plot_graphs)

In [ ]:
my_test

In [39]:
date_data=df.groupby(by=['store_nbr','family'])['date'].agg(['min','max']).reset_index()

In [40]:
date_data['diff']=np.where(date_data['min']!=date_data['max'],1,0)

In [ ]:
date_data['min'].unique(),date_data['max'].unique()

In [ ]:
date_data

In [43]:
# df['default_rank'] = df.groupby(['store_nbr', 'family']).cumcount() + 1

In [ ]:
df

In [45]:
unique_data=df.groupby(by=['store_nbr','family'])['date'].agg(['min','max']).reset_index()

In [46]:
unique_data['default_rank'] = unique_data.index+1

In [47]:
unique_data['comb']=unique_data['store_nbr'].astype(str)+str('_')+unique_data['family'].astype(str)

In [ ]:
unique_data

In [49]:
df['comb']=df['store_nbr'].astype(str)+str('_')+df['family'].astype(str)

In [50]:
df=unique_data.merge(df,on=['comb'],how='right')

In [ ]:
df.columns

In [52]:
df=df[['store_nbr_x', 'family_x', 'min', 'max', 'default_rank', 'comb', 'id',
       'date', 'sales', 'onpromotion', 'city',
       'state', 'store_type', 'cluster', 'transactions']].rename(columns={'store_nbr_x':'store_nbr', 'family_x':'family'})

In [ ]:
len(df['default_rank'].unique())

In [ ]:
df

In [55]:
def filter_data_for_group(df,groupno,group_value=None):
    if groupno:
        new_data=df[df['default_rank']==groupno]
    elif group_value:
        new_data=df[df['comb']==group_value]
    return new_data

In [ ]:
filter_data_for_group(df=df,groupno=56)

In [ ]:
filter_data_for_group(df=df,groupno=56).groupby(by=['store_nbr','family']).apply(plot_graphs)

In [ ]:
df.info()

In [ ]:
holiday.info()
holiday.set_index('date',inplace=True)

In [ ]:
holiday.index

In [ ]:
holiday.type.unique()

In [62]:
def filter_data_year(data, year):
    # print(type(year))
    
    # Check if the year is a list with two elements
    if isinstance(year, list):  # Use isinstance instead of type() to check list
        data = data[(data.index.year >= year[0]) & (data.index.year <= year[1])]  # Use & instead of and for element-wise comparison
        # print('using list filter')
    else:
        data = data[data.index.year >= year]
    
    return data

def filter_data_month(data, month):
    # print(type(year))
    
    # Check if the year is a list with two elements
    if isinstance(month, list):  # Use isinstance instead of type() to check list
        data = data[(data.index >= month[0]) & (data.index <= month[1])]  # Use & instead of and for element-wise comparison
        # print('using list filter')
    else:
        data = data[data.index >= month]
    
    return data


In [ ]:
holiday['category'].unique()

In [ ]:
holiday_Local

In [ ]:
df.head()

In [66]:
df_hol=df.merge(holiday_Local.reset_index(),left_on=['date','city'],right_on=['date','locale_name'],how='left')
df_hol['local_holiday']=np.where(df_hol['locale_name'].isna(),False,True)
df_hol=df_hol[df.columns.to_list()+['local_holiday']]
df_hol=df_hol.merge(holiday_Regional.reset_index(),left_on=['date','state'],right_on=['date','locale_name'],how='left')
df_hol['regional_holiday']=np.where(df_hol['locale_name'].isna(),False,True)
df_hol=df_hol[df.columns.to_list()+['local_holiday','regional_holiday']]
df_hol=df_hol.merge(holiday_national.reset_index(),left_on=['date'],right_on=['date'],how='left')
df_hol['national_holiday']=np.where(df_hol['locale_name'].isna(),False,True)
# df_hol=df_hol[df.columns.to_list()+['local_holiday','regional_holiday','national_holiday']]
df_hol['holiday_true']=df_hol['local_holiday'] | df_hol['regional_holiday'] |df_hol['national_holiday']

In [67]:
# # df_hol['_merge'].unique()
# # df_hol.to_csv('sales_holiday.csv',index=False)
# df_hol.set_index('date').index.year.unique().to_list()

In [ ]:
df_hol.head()

In [69]:
# pd.notna(df_hol[df_hol['_merge']=='left_only']['store_nbr']).unique()

In [ ]:
df_hol.info()

In [ ]:
# data=df_hol[df_hol['category_reg']=='Dia'].set_index('date')
data=df_hol.set_index('date')
data=data[data.index.year == 2017]
data=data[data['holiday_true']==True]
data=data[data['default_rank']==55]
data.shape

In [ ]:
year_range=[2016,2016]

# Filter data to include only rows for years greater than 2015
filtered_data = filter_data_for_group(df=df_hol, groupno=9).set_index('date')

# Resample the data by month (MS - Month Start) and sum the values for 'sales' and 'onpromotion'
filtered_data.resample('MS')['sales', 'onpromotion'].sum()
filtered_data = filter_data_year(filtered_data,year_range)  # Filter by year

filtered_data.plot(y=['sales', 'onpromotion'], figsize=(15, 13))


unique_colors = ['red', 'green', 'blue', 'orange', 'purple', 'pink', 'brown']  # Replace with your actual colors list

# # Create a dictionary for color-to-label mapping
# color_to_label = {color: hol_day for color, hol_day in zip(unique_colors, holiday['category_reg'].unique())}

# for color, hol_day in zip(unique_colors, holiday['category_reg'].unique()):
#     holiday_data = holiday[holiday['category_reg'] == hol_day]
#     holiday_data = filter_data_year(holiday_data,year_range)
#     for date in holiday_data.index:
#         # Plot each vertical line and assign a label to the color
#         plt.axvline(x=date, color=color, linestyle='--', label=color_to_label[color] if date == holiday_data.index[0] else "")

# # Display the legend
# plt.legend(title='Holiday Types', loc='upper right')
filtered_holiday_data=filtered_data[filtered_data['holiday_true']==True]
color_to_label = {color: hol_day for color, hol_day in zip(unique_colors, filtered_holiday_data['category_reg'].unique())}
# print(color_to_label)

for color, hol_day in zip(unique_colors, filtered_holiday_data['category_reg'].unique()):
    filtered_holiday = filtered_holiday_data[filtered_holiday_data['category_reg'] == hol_day]
    # print(filtered_holiday_data)
    # print(filtered_holiday_data.shape)
    for date in filtered_holiday.index:
        # print(date)
        plt.axvline(x=date, color=color, linestyle='--', label=color_to_label[color]  if date == filtered_holiday.index[0] else "")
    
plt.legend(title='Holiday Types', loc='upper right')
plt.show()

In [ ]:
year_range=[2016,2016]
month_range=['2016-01-01','2016-12-31']
filtered_data=filter_data_for_group(df=df_hol,groupno=9).set_index('date')
filtered_data=filter_data_month(filtered_data,month_range)
filtered_data.plot(y=['sales','onpromotion'],figsize=(15,13))


unique_colors = ['red', 'green', 'blue', 'orange', 'purple', 'pink', 'brown']  # Replace with your actual colors list

# # Create a dictionary for color-to-label mapping
# color_to_label = {color: hol_day for color, hol_day in zip(unique_colors, holiday['category_reg'].unique())}

# for color, hol_day in zip(unique_colors, holiday['category_reg'].unique()):
#     holiday_data = holiday[holiday['category_reg'] == hol_day]
#     holiday_data = filter_data_year(holiday_data,year_range)
#     for date in holiday_data.index:
#         # Plot each vertical line and assign a label to the color
#         plt.axvline(x=date, color=color, linestyle='--', label=color_to_label[color] if date == holiday_data.index[0] else "")

# # Display the legend
# plt.legend(title='Holiday Types', loc='upper right')

filtered_holiday_data=filtered_data[filtered_data['holiday_true']==True]
color_to_label = {color: hol_day for color, hol_day in zip(unique_colors, filtered_holiday_data['category_reg'].unique())}
# print(color_to_label)
count=0
for color, hol_day in zip(unique_colors, filtered_holiday_data['category_reg'].unique()):
    filtered_holiday = filtered_holiday_data[filtered_holiday_data['category_reg'] == hol_day]
    # print(filtered_holiday)
    # print(filtered_holiday.shape)
    for date in filtered_holiday.index:
        # print(date)
        
        plt.axvline(x=date, color=color, linestyle='--', label=color_to_label[color]  if date == filtered_holiday.index[0] else "")
        count=count+1
plt.title(f"sales vs holiday : {count}")  
plt.legend(title='Holiday Types', loc='upper right')
plt.show()

In [74]:
for i in range(10):
    groupno=random.randint(1,100)
    

In [ ]:
for i in range(2):
    groupno=random.randint(1,100)
    month_range=['2017-01-01','2017-12-31']
    filtered_data=filter_data_for_group(df=df_hol,groupno=groupno).set_index('date')
    filtered_data=filter_data_month(filtered_data,month_range)
    filtered_data.plot(y=['sales','onpromotion'],figsize=(15,13))
    
    
    unique_colors = ['red', 'green', 'blue', 'orange', 'purple', 'pink', 'brown']  # Replace with your actual colors list
    
    # # Create a dictionary for color-to-label mapping
    # color_to_label = {color: hol_day for color, hol_day in zip(unique_colors, holiday['category_reg'].unique())}
    
    # for color, hol_day in zip(unique_colors, holiday['category_reg'].unique()):
    #     holiday_data = holiday[holiday['category_reg'] == hol_day]
    #     holiday_data = filter_data_year(holiday_data,year_range)
    #     for date in holiday_data.index:
    #         # Plot each vertical line and assign a label to the color
    #         plt.axvline(x=date, color=color, linestyle='--', label=color_to_label[color] if date == holiday_data.index[0] else "")
    
    # # Display the legend
    # plt.legend(title='Holiday Types', loc='upper right')
    filtered_holiday_data=filtered_data[filtered_data['holiday_true']==True]
    color_to_label = {color: hol_day for color, hol_day in zip(unique_colors, filtered_holiday_data['category_reg'].unique())}
    # print(color_to_label)
    count=0
    for color, hol_day in zip(unique_colors, filtered_holiday_data['category_reg'].unique()):
        filtered_holiday = filtered_holiday_data[filtered_holiday_data['category_reg'] == hol_day]
        # print(filtered_holiday)
        # print(filtered_holiday.shape)
        for date in filtered_holiday.index:
            # print(date)
            
            plt.axvline(x=date, color=color, linestyle='--', label=color_to_label[color]  if date == filtered_holiday.index[0] else "")
            count=count+1
    plt.title(f"sales vs holiday : {count} for the Group {groupno}" )  
    plt.legend(title='Holiday Types', loc='upper right')
    plt.show()

In [ ]:
year_range=[2016,2016]

# Filter data to include only rows for years greater than 2015
filtered_data = filter_data_for_group(df=df, groupno=56).set_index('date')

# Resample the data by month (MS - Month Start) and sum the values for 'sales' and 'onpromotion'
filtered_data.resample('MS')['sales', 'onpromotion'].sum()
filtered_data = filter_data_year(filtered_data,year_range)  # Filter by year

filtered_data.plot(y=['sales', 'onpromotion'], figsize=(15, 13))


unique_colors = ['red', 'green', 'blue', 'orange', 'purple', 'pink', 'brown']  # Replace with your actual colors list

# Create a dictionary for color-to-label mapping
color_to_label = {color: hol_day for color, hol_day in zip(unique_colors, holiday['category_reg'].unique())}
count=0
for color, hol_day in zip(unique_colors, holiday['category_reg'].unique()):
    holiday_data = holiday[holiday['category_reg'] == hol_day]
    holiday_data = filter_data_year(holiday_data,year_range)
    for date in holiday_data.index:
        # Plot each vertical line and assign a label to the color
        plt.axvline(x=date, color=color, linestyle='--', label=color_to_label[color] if date == holiday_data.index[0] else "")
        count=count+1

# Display the legend
plt.legend(title='Holiday Types', loc='upper right')
plt.title(f"sales vs holiday : {count}")
plt.show()



In [77]:
# fdata[fdata.index>'2015-01-05']

In [78]:
# month_range=['2015-01-05','2015-03-05']
# filter_data_month(fdata,month_range)

In [ ]:
month_range=['2016-04-05','2016-07-05']
fdata=filter_data_for_group(df=df,groupno=76).set_index('date')
fdata=filter_data_month(fdata,month_range)
fdata.plot(y=['sales','onpromotion'],figsize=(15,13))

unique_colors = ['red', 'green', 'blue', 'orange', 'purple', 'pink', 'brown']  # Replace with your actual colors list

# Create a dictionary for color-to-label mapping
color_to_label = {color: hol_day for color, hol_day in zip(unique_colors, holiday['category_reg'].unique())}

for color, hol_day in zip(unique_colors, holiday['category_reg'].unique()):
    holiday_data = holiday[holiday['category_reg'] == hol_day]
    holiday_data=filter_data_month(holiday_data,month_range)
    for date in holiday_data.index:
        # Plot each vertical line and assign a label to the color
        plt.axvline(x=date, color=color, linestyle='--', label=color_to_label[color] if date == holiday_data.index[0] else "")

# Display the legend
plt.legend(title='Holiday Types', loc='upper right')

plt.show()

In [80]:
def Split_data(df,groupno):
    filter_data=filter_data_for_group(df=df,groupno=groupno)
    train_cuttoff=60
    train_data=filter_data[:-train_cuttoff]
    test_data=filter_data[-train_cuttoff:]
    return train_data,test_data,filter_data

In [81]:
def ComplexProphetPlot(model,forecast,train,test,df):
    fig=plot(m=model, fcst=forecast, ax=None, uncertainty=True, plot_cap=True, xlabel='dates', ylabel='y_pred',
    figsize=(10, 6), include_legend=True);
    a=add_changepoints_to_plot(fig.gca(),m=model,fcst=forecast)
    train.set_index('ds').plot(ax=fig.gca(),legend=True,ls='--',c='red')
    test.set_index('ds').rename(columns={'y':'y_true'}).plot(ax=fig.gca(),legend=True,ls='dashdot',c='black')
    #df.set_index('ds').rename(columns={'y':'y_full_data'}).plot(ax=fig.gca(),legend=True,ls='dotted',c='black',marker='*')

In [82]:
def ProphetModel(df,groupno):
    train_data,test_data,all_data=Split_data(df,groupno)
    train_data=train_data[['date','sales','onpromotion','holiday_true']]
    test_data=test_data[['date','sales','onpromotion','holiday_true']]
    train_data.rename(columns={'date':'ds','sales':'y'},inplace=True)
    test_data.rename(columns={'date':'ds','sales':'y'},inplace=True)
    
    scaler_sales = MinMaxScaler()
    scaler_onpromotion = MinMaxScaler()
    
    train_data['y'] = scaler_sales.fit_transform(train_data[['y']])
    test_data['y'] = scaler_sales.transform(test_data[['y']])
    
    train_data['onpromotion'] = scaler_onpromotion.fit_transform(train_data[['onpromotion']])
    test_data['onpromotion'] = scaler_onpromotion.transform(test_data[['onpromotion']])

    
    test_data.to_csv('test_data.csv',index=False)
    model=Prophet(daily_seasonality=True)
    model.add_regressor('holiday_true')
    model.add_regressor('onpromotion')
    model.fit(train_data)
    future_dfs=model.make_future_dataframe(periods=60)
    future_dfs.to_csv('future_data.csv',index=False)
    future_dfs = future_dfs.merge(
    test_data[['ds', 'holiday_true', 'onpromotion']],
    on='ds',
    how='left'
    )
    future_dfs.fillna({'holiday_true': 0, 'onpromotion': 0}, inplace=True)

    future_dfs=future_dfs.tail(60).head(60)
    future_dfs.to_csv('future_dfs.csv',index=False)
    preds=model.predict(future_dfs)
    # preds.merge(train_data,on='ds',how='left').set_index('ds')[['y','yhat']].plot(figsize=(15,13))
    #ComplexProphetPlot(model,preds,train_data,test_data,df)
    preds['yhat'] = scaler_sales.inverse_transform(preds[['yhat']])
    preds['yhat_lower'] = scaler_sales.inverse_transform(preds[['yhat_lower']])
    preds['yhat_upper'] = scaler_sales.inverse_transform(preds[['yhat_upper']])
    
    # Inverse transform test data to original scale for evaluation
    test_data['y'] = scaler_sales.inverse_transform(test_data[['y']])
    
    return preds,train_data,test_data

In [83]:
# ProphetModel(df_hol,groupno=88)

In [ ]:
req_columns=['default_rank','comb','date','sales','onpromotion','holiday_true']
df_Prophet=df_hol[req_columns]
# df_Prophet=df_Prophet.set_index('date')
df_Prophet.loc[:,'holiday_true']=df_Prophet['holiday_true'].astype('int64')
df_Prophet

In [85]:
def fillMissingDates(df_df,unique_cols,frequency,freq,grain_with_time,sales_column,date_index=False):
    date_ranges = df_df.groupby(unique_cols).apply(
            lambda group: pd.date_range(group[frequency].min(), group[frequency].max(), freq=freq)
        ).reset_index(name=frequency)


    # Step 4: Create a new DataFrame that combines item-location combinations with the generated date ranges
    date_ranges = date_ranges.explode(frequency).reset_index(drop=True)


    # Step 5: Merge the generated date ranges with the original data to get sales_quantity for each date
    df_filled = pd.merge(date_ranges, df_df[grain_with_time+sales_column],
                         on=unique_cols+[frequency], how='left')

    # Step 6: Fill missing sales_quantity values with 0
    df_filled[sales_column] = df_filled[sales_column].fillna(0)
    if date_index:
        return df_filled
    # Step 7: Optionally, reset index if you want
    else:
        df_filled.reset_index(drop=True, inplace=True)
        return df_filled

In [86]:
df_Prophet=fillMissingDates(df_Prophet,unique_cols=['default_rank'],frequency='date',freq='D',grain_with_time=['default_rank','date'],sales_column=['sales','onpromotion','holiday_true'])

In [ ]:
df_Prophet

In [88]:
# scaler=MinMaxScaler()
# df_Prophet.loc[:,'sales']=scaler.fit_transform(df_Prophet[['sales']])
# df_Prophet.loc[:,'onpromotion']=scaler.fit_transform(df_Prophet[['onpromotion']])

In [89]:
df_Prophet=df_Prophet.fillna(0)

In [ ]:
pred_data,train_data,test_data=ProphetModel(df_Prophet,groupno=88)

In [ ]:
pred_data.tail(60).head()

In [ ]:
test_data.head()

In [93]:
pred_data_prophet=test_data[['ds','y']].merge(pred_data[['ds','yhat','yhat_lower','yhat_upper']],on='ds',how='left')

In [ ]:
pred_data_prophet.head()

In [ ]:
np.sqrt(mean_squared_error(pred_data_prophet['y'],pred_data_prophet['yhat']))

In [ ]:
np.sqrt(mean_squared_error(pred_data_prophet['y'],pred_data_prophet['yhat_lower']))

In [ ]:
np.sqrt(mean_squared_error(pred_data_prophet['y'],pred_data_prophet['yhat_upper']))

In [98]:
import numpy as np

def rmsle(y_true, y_pred):
    """
    Calculate Root Mean Squared Logarithmic Error (RMSLE).

    Parameters:
    - y_true (array-like): Actual values (ground truth).
    - y_pred (array-like): Predicted values.

    Returns:
    - float: RMSLE value.
    """
    # Ensure no negative predictions
    y_pred = np.maximum(y_pred, 0)
    
    # Apply the formula
    log_true = np.log1p(y_true)
    log_pred = np.log1p(y_pred)
    squared_log_error = np.square(log_true - log_pred)
    rmsle_value = np.sqrt(np.mean(squared_log_error))
    
    return rmsle_value


In [ ]:
rmsle(pred_data_prophet['y'],pred_data_prophet['yhat_upper']),rmsle(pred_data_prophet['y'],pred_data_prophet['yhat'])

In [ ]:
px.line(pred_data_prophet,x='ds',y=pred_data_prophet.columns)

In [101]:
def season_plots(data):
    seasonal_data=seasonal_decompose(x=data,
    model="additive",
    filt=None,
    period=None,
    two_sided=True,
    extrapolate_trend=0,
)
    seasonal_data.plot()
    return seasonal_data.trend,seasonal_data.seasonal,seasonal_data.resid

In [102]:
groupno=10
data=filter_data_for_group(df=df_hol,groupno=groupno).set_index('date')[['sales']]

In [103]:
data=data.resample('D').sum()


In [ ]:
trend_14,season_14,resid_14=season_plots(data[data.index.year==2014]);
trend_15,season_15,resid_15=season_plots(data[data.index.year==2015]);
trend_16,season_16,resid_16=season_plots(data[data.index.year==2016]);
trend_17,season_17,resid_17=season_plots(data[data.index.year==2017]);

In [ ]:
type(trend_14)

In [106]:
trend_data_14=trend_14.reset_index().rename(columns={'trend':'trend_14'})
trend_data_15=trend_15.reset_index().rename(columns={'trend':'trend_15'})
trend_data_16=trend_16.reset_index().rename(columns={'trend':'trend_16'})
trend_data_17=trend_17.reset_index().rename(columns={'trend':'trend_17'})

In [107]:
trend_data_15['date'] = trend_data_15['date'] - pd.DateOffset(years=1)
trend_data_16['date'] = trend_data_16['date'] - pd.DateOffset(years=2)
trend_data_17['date'] = trend_data_17['date'] - pd.DateOffset(years=3)

In [108]:
trend_data=trend_data_15.merge(trend_data_14,on='date',how='left').merge(trend_data_16,on='date',how='left').merge(trend_data_17,on='date',how='left')

In [ ]:
px.line(trend_data,x='date',y=['trend_14','trend_15','trend_16','trend_17'])

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(trend_14, label='2014 Trend', color='blue', linestyle='-', linewidth=2)
plt.plot(trend_15, label='2015 Trend', color='orange', linestyle='--', linewidth=2)
plt.legend()
plt.show()

In [ ]:
deviation = trend_data['trend_15'] - trend_data['trend_14']
deviation.plot(kind='line', title='Deviation between 2014 and 2015 Trends')
plt.show()

In [112]:
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima.model import ARIMA, ARIMAResults

In [113]:
def MyAutoArima(data,x=None):
    arima_model=auto_arima(y=data,
    X=x,
    start_p=2,
    d=None,
    start_q=2,
    max_p=10,
    max_d=10,
    max_q=10,
    start_P=1,
    D=None,
    start_Q=1,
    max_P=10,
    max_D=10,
    max_Q=10,
    max_order=5,
    m=1,
    seasonal=True,
    stationary=False,
    information_criterion='aic',
    alpha=0.05,
    test='kpss',
    seasonal_test='ocsb',
    stepwise=False,
    n_jobs=-1,
    start_params=None,
    trend=None,
    method='lbfgs',
    maxiter=500,
    offset_test_args=None,
    seasonal_test_args=None,
    suppress_warnings=True,
    error_action='trace',
    trace=False,
    random=False,
    random_state=None,
    n_fits=100,
    return_valid_fits=False,
    out_of_sample_size=0,
    scoring='mse',
    scoring_args=None,
    with_intercept="auto")
    return arima_model

In [ ]:
req_columns=['default_rank','comb','date','sales','onpromotion','holiday_true']
df_arima_main=df_hol[req_columns]
# df_Prophet=df_Prophet.set_index('date')
df_arima_main.loc[:,'holiday_true']=df_arima_main['holiday_true'].astype('int64')
df_arima_main

In [115]:
df_arima=fillMissingDates(df_arima_main,unique_cols=['default_rank'],frequency='date',freq='D',grain_with_time=['default_rank','date'],sales_column=['sales','onpromotion','holiday_true'])

In [ ]:
df_arima

In [ ]:
groupno=45
train_data,test_data,alll_data=Split_data(df=df_arima,groupno=groupno)
# arima_model=MyAutoArima(data)
arima_scaler_sales = MinMaxScaler()
arima_scaler_onpromotion = MinMaxScaler()

train_data['sales'] = arima_scaler_sales.fit_transform(train_data[['sales']])
test_data['sales'] = arima_scaler_sales.transform(test_data[['sales']])

train_data['onpromotion'] = arima_scaler_onpromotion.fit_transform(train_data[['onpromotion']])
test_data['onpromotion'] = arima_scaler_onpromotion.transform(test_data[['onpromotion']])


train=train_data.set_index('date')[['sales']]
x=train_data.set_index('date')[['holiday_true','onpromotion']]
x_test=test_data.set_index('date')[['holiday_true','onpromotion']]



In [118]:

# Resample to daily frequency to fill missing dates


In [ ]:
train['group']=1
train.reset_index(inplace=True)
train=fillMissingDates(train,unique_cols=['group'],frequency='date',freq='D',grain_with_time=['group','date'],sales_column=['sales'])
train.set_index('date',inplace=True)
train.index = pd.to_datetime(train.index)
if train.index.duplicated().any():
    print("Duplicate entries found. Resolving...")
    train = train[~train.index.duplicated(keep='first')]
train.index.freq='D'
train=train[['sales']]

In [ ]:
x['group']=1
x.reset_index(inplace=True)
x=fillMissingDates(x,unique_cols=['group'],frequency='date',freq='D',grain_with_time=['group','date'],sales_column=['holiday_true','onpromotion'])
x.set_index('date',inplace=True)
x.index = pd.to_datetime(x.index)
if x.index.duplicated().any():
    print("Duplicate entries found. Resolving...")
    x = x[~x.index.duplicated(keep='first')]
x.index.freq='D'
x=x[['holiday_true','onpromotion']]

In [121]:
if train.index.duplicated().any():
    print("Duplicate entries found. Resolving...")
    train = train[~train.index.duplicated(keep='first')]


In [ ]:
train

In [123]:
# arima_model=MyAutoArima(train,x)
# arima_model

In [124]:
# arima_model

In [ ]:
x

In [ ]:
train.columns

In [ ]:
test_data.columns

In [ ]:
data=train
data.shape,x.shape

In [129]:

arima_model=ARIMA(endog=data,                  
      exog=x,
      order=(7, 1, 7), 
      seasonal_order=(0, 0, 0, 0), 
      trend=None,
      enforce_stationarity=True, 
      enforce_invertibility=True,
      concentrate_scale=False, 
      trend_offset=1, 
      dates=None,
      freq=None,
      missing='none', 
      validate_specification=True)

In [ ]:
res = arima_model.fit()

In [131]:
end=len(train)+len(test_data)-1
start=len(train)
pred_data=res.predict(start=len(train),end=len(train)+len(test_data)-1,exog=x_test).rename('Predicted_717V2_group88')


In [ ]:
predicted_df=pd.DataFrame(pred_data)
predicted_df.head()

In [133]:
predicted_df=predicted_df.reset_index().rename(columns={'index':'date'})

In [ ]:
predicted_df['Predicted_717V2_group88']=arima_scaler_sales.inverse_transform(predicted_df[['Predicted_717V2_group88']])
test_data['sales']=arima_scaler_sales.inverse_transform(test_data[['sales']])

In [ ]:
predicted_df.head()

In [136]:

test_pred=test_data[['date','sales']].merge(predicted_df,on='date',how='left')


In [ ]:
test_pred.set_index('date').plot()
plt.show()

In [ ]:
px.line(test_pred,x='date',y=test_pred.columns)

In [139]:
# np.sqrt(mean_squared_error(test_pred['sales'],test_pred['Predicted_717V2_y']))

In [ ]:
rmsle(test_pred['sales'],test_pred['Predicted_717V2_group88'])

In [141]:
# my_test=pd.read_csv('data/test.csv',parse_dates=['date'])
# oil=pd.read_csv('data/oil.csv',parse_dates=['date'])
# train=pd.read_csv('data/train.csv',parse_dates=['date']) # main df # main df

# stores=pd.read_csv('data/stores.csv') # can be merged with train - on store no
# stores.rename(columns={'type':'store_type'},inplace=True)
# ts=pd.read_csv('data/transactions.csv',parse_dates=['date']) # can be merged with train - on store no and date

In [142]:
data='/kaggle/input/store-sales-time-series-forecasting'

In [143]:
my_test=pd.read_csv(f'{data}/test.csv',parse_dates=['date'])
oil=pd.read_csv(f'{data}/oil.csv',parse_dates=['date'])
train=pd.read_csv(f'{data}/train.csv',parse_dates=['date']) # main df # main df

stores=pd.read_csv(f'{data}/stores.csv') # can be merged with train - on store no
stores.rename(columns={'type':'store_type'},inplace=True)
ts=pd.read_csv(f'{data}/transactions.csv',parse_dates=['date']) # can be merged with train - on store no and date

In [ ]:
ts

In [145]:
rand_comb=df_hol[['default_rank','comb']].drop_duplicates()

In [ ]:
test_data=my_test.copy(deep=True)
test_data

In [147]:
test_data['comb']=test_data['store_nbr'].astype(str)+'_'+test_data['family']
test_data=test_data.merge(rand_comb,on='comb',how='outer',indicator=True)


In [ ]:
test_data['_merge'].unique()

In [ ]:
test_data=test_data.merge(stores,on='store_nbr',how='left')
test_data

In [150]:
# df_hol=df.merge(holiday_Local.reset_index(),left_on=['date','city'],right_on=['date','locale_name'],how='left')
# df_hol['local_holiday']=np.where(df_hol['locale_name'].isna(),False,True)
# df_hol=df_hol[df.columns.to_list()+['local_holiday']]
# df_hol=df_hol.merge(holiday_Regional.reset_index(),left_on=['date','state'],right_on=['date','locale_name'],how='left')
# df_hol['regional_holiday']=np.where(df_hol['locale_name'].isna(),False,True)
# df_hol=df_hol[df.columns.to_list()+['local_holiday','regional_holiday']]
# df_hol=df_hol.merge(holiday_national.reset_index(),left_on=['date'],right_on=['date'],how='left')
# df_hol['national_holiday']=np.where(df_hol['locale_name'].isna(),False,True)
# # df_hol=df_hol[df.columns.to_list()+['local_holiday','regional_holiday','national_holiday']]
# df_hol['holiday_true']=df_hol['local_holiday'] | df_hol['regional_holiday'] |df_hol['national_holiday']

In [151]:
test_data_hol=test_data.merge(holiday_Local.reset_index(),left_on=['date','city'],right_on=['date','locale_name'],how='left')
test_data_hol['local_holiday']=np.where(test_data_hol['locale_name'].isna(),False,True)
test_data_hol=test_data_hol[test_data.columns.to_list()+['local_holiday']]
test_data_hol=test_data_hol.merge(holiday_Regional.reset_index(),left_on=['date','state'],right_on=['date','locale_name'],how='left')
test_data_hol['regional_holiday']=np.where(test_data_hol['locale_name'].isna(),False,True)
test_data_hol=test_data_hol[test_data.columns.to_list()+['local_holiday','regional_holiday']]
test_data_hol=test_data_hol.merge(holiday_national.reset_index(),left_on=['date'],right_on=['date'],how='left')
test_data_hol['national_holiday']=np.where(test_data_hol['locale_name'].isna(),False,True)
# df_hol=df_hol[df.columns.to_list()+['local_holiday','regional_holiday','national_holiday']]
test_data_hol['holiday_true']=test_data_hol['local_holiday'] | test_data_hol['regional_holiday'] |test_data_hol['national_holiday']

In [ ]:
test_data_hol

In [153]:
def Train_Predict(train,test,groupno):
    # filter the train and test data
    train_data=filter_data_for_group(train,groupno)
    test_data=filter_data_for_group(test,groupno)

    #filter required columns
    req_columns_train=['default_rank','comb','date','sales','onpromotion','holiday_true']
    req_columns_test=['default_rank','comb','date','onpromotion','holiday_true']
    train_data_f=train_data[req_columns_train]
    test_data_f=test_data[req_columns_test]

    # change holiday data into ints
    train_data_f.loc[:,'holiday_true']=train_data_f['holiday_true'].astype('int64')
    test_data_f.loc[:,'holiday_true']=test_data_f['holiday_true'].astype('int64')

    #fill missing dates
    train_data_ff=fillMissingDates(train_data_f,unique_cols=['default_rank'],frequency='date',freq='D',grain_with_time=['default_rank','date'],sales_column=['sales','onpromotion','holiday_true'])
    test_data_ff=fillMissingDates(test_data_f,unique_cols=['default_rank'],frequency='date',freq='D',grain_with_time=['default_rank','date'],sales_column=['onpromotion','holiday_true'])


    # min max scaler

    arima_scaler_sales = MinMaxScaler()
    arima_scaler_onpromotion = MinMaxScaler()
    
    train_data_ff['sales'] = arima_scaler_sales.fit_transform(train_data_ff[['sales']])
    # test_data_ff['sales'] = arima_scaler_sales.transform(test_data_ff[['sales']])
    
    train_data_ff['onpromotion'] = arima_scaler_onpromotion.fit_transform(train_data_ff[['onpromotion']])
    test_data_ff['onpromotion'] = arima_scaler_onpromotion.transform(test_data_ff[['onpromotion']])
    
    
    train_data_fff=train_data_ff.set_index('date')[['sales']]
    x=train_data_ff.set_index('date')[['holiday_true','onpromotion']]
    
    # handle the frq of the train and test data and exog variables data   
    train_data_fff['group']=1
    train_data_fff.reset_index(inplace=True)
    train_data_ffff=fillMissingDates(train_data_fff,unique_cols=['group'],frequency='date',freq='D',grain_with_time=['group','date'],sales_column=['sales'])
    train_data_ffff.set_index('date',inplace=True)
    train_data_ffff.index = pd.to_datetime(train_data_ffff.index)
    if train_data_ffff.index.duplicated().any():
        print("Duplicate entries found. Resolving...")
        train_data_ffff = train_data_ffff[~train_data_ffff.index.duplicated(keep='first')]
    train_data_ffff.index.freq='D'
    train_data_ffff=train_data_ffff[['sales']]
    

    x['group']=1
    x.reset_index(inplace=True)
    x=fillMissingDates(x,unique_cols=['group'],frequency='date',freq='D',grain_with_time=['group','date'],sales_column=['holiday_true','onpromotion'])
    x.set_index('date',inplace=True)
    x.index = pd.to_datetime(x.index)
    if x.index.duplicated().any():
        print("Duplicate entries found. Resolving...")
        x = x[~x.index.duplicated(keep='first')]
    x.index.freq='D'
    x=x[['holiday_true','onpromotion']]

    
    x_test=test_data_ff.set_index('date')[['holiday_true','onpromotion']]

    arima_model=MyAutoArima(train_data_ffff,x)
    res=arima_model.fit(y=train_data_ffff,X=x)
    
    start = len(train_data_ffff)
    end = start + len(x_test) - 1
    
    assert x.shape[1] == x_test.shape[1], "Mismatch in number of columns between training and test exogenous variables."
    assert len(x_test) == (end - start + 1), "Mismatch between prediction period and test exogenous variables."



    
    
    # Predict with exogenous variables
    pred_data = res.predict(n_periods=len(x_test), X=x_test).rename('sales')


    predicted_df=pd.DataFrame(pred_data)

    predicted_df['sales']=arima_scaler_sales.inverse_transform(predicted_df[['sales']]) 

    predicted_df=predicted_df.reset_index().rename(columns={'index':'date'})

    predicted_df=test_data[['id','date']].merge(predicted_df,on='date',how='left')
    predicted_df['Group']=groupno
    
    # # Output predictions
    return res,x_test,end,start,predicted_df,train_data_f,test_data
    

In [ ]:
res,x_test,end,start,predicted_df,train_data_f,test_data=Train_Predict(df_hol,test_data_hol,77)

In [155]:
# ??res.predict


In [ ]:
predicted_df

In [ ]:
all_groups=test_data_hol['default_rank'].unique()
all_groups

In [158]:
# all_predicted_data=[]
# for group in all_groups:
#     res,x_test,end,start,predicted_df,train_data_f,test_data=Train_Predict(df_hol,test_data_hol,group)
#     all_predicted_data.append(predicted_df)

In [159]:
def Train_Predict_Prophet(train,test,groupno):
    # filter the train and test data
    train_data=filter_data_for_group(train,groupno)
    test_data=filter_data_for_group(test,groupno)

    #filter required columns
    req_columns_train=['default_rank','comb','date','sales','onpromotion','holiday_true']
    req_columns_test=['default_rank','comb','date','onpromotion','holiday_true']
    train_data_f=train_data[req_columns_train]
    test_data_f=test_data[req_columns_test]

    # change holiday data into ints
    train_data_f.loc[:,'holiday_true']=train_data_f['holiday_true'].astype('int64')
    test_data_f.loc[:,'holiday_true']=test_data_f['holiday_true'].astype('int64')

    #fill missing dates
    train_data_ff=fillMissingDates(train_data_f,unique_cols=['default_rank'],frequency='date',freq='D',grain_with_time=['default_rank','date'],sales_column=['sales','onpromotion','holiday_true'])
    test_data_ff=fillMissingDates(test_data_f,unique_cols=['default_rank'],frequency='date',freq='D',grain_with_time=['default_rank','date'],sales_column=['onpromotion','holiday_true'])


    # min max scaler

    Prophet_scaler_sales = MinMaxScaler()
    Prophet_scaler_onpromotion = MinMaxScaler()
    
    train_data_ff['sales'] = Prophet_scaler_sales.fit_transform(train_data_ff[['sales']])
    # test_data_ff['sales'] = Prophet_scaler_sales.transform(test_data_ff[['sales']])
    
    train_data_ff['onpromotion'] = Prophet_scaler_onpromotion.fit_transform(train_data_ff[['onpromotion']])
    test_data_ff['onpromotion'] = Prophet_scaler_onpromotion.transform(test_data_ff[['onpromotion']])

    # convert_columns into standard names

    train_data_ff.rename(columns={'date':'ds','sales':'y'},inplace=True)
    test_data_ff.rename(columns={'date':'ds'},inplace=True)
    print(train_data_ff.columns)
    
    # define the model

    model=Prophet(daily_seasonality=True)
    # model.add_regressor('holiday_true')
    # model.add_regressor('onpromotion')
    model.fit(train_data_ff)
    future_dfs=model.make_future_dataframe(periods=len(test_data_ff))
    
    
    future_dfs = future_dfs.merge(
    test_data_ff[['ds', 'holiday_true', 'onpromotion']],
    on='ds',
    how='left'
    )
    future_dfs.fillna({'holiday_true': 0, 'onpromotion': 0}, inplace=True)



    
    # Predict with exogenous variables
    future_dfs=future_dfs.tail(len(test_data_ff)).head(len(test_data_ff))
    preds=model.predict(future_dfs)

    preds['yhat'] = Prophet_scaler_sales.inverse_transform(preds[['yhat']])
    preds['yhat_lower'] = Prophet_scaler_sales.inverse_transform(preds[['yhat_lower']])
    preds['yhat_upper'] = Prophet_scaler_sales.inverse_transform(preds[['yhat_upper']])
    
    #
    
    predicted_df=preds[['ds','yhat','yhat_upper','yhat_lower']]
    predicted_df=test_data[['id','date']].merge(predicted_df,right_on='ds',left_on='date',how='left')
    predicted_df['Group']=groupno
    
    # # Output predictions
    return model,future_dfs,predicted_df,train_data_f,test_data
    

In [160]:
import pandas as pd
import os
from tqdm import tqdm

# Function to save data periodically
def save_batch_data(data_list, save_path, batch_number):
    if not os.path.exists(save_path):
        os.makedirs(save_path)  # Create directory if it doesn't exist
    
    combined_df = pd.concat(data_list, ignore_index=True)
    file_name = f"predicted_batch_prophet_v1_{batch_number}.csv"
    file_path = os.path.join(save_path, file_name)
    combined_df.to_csv(file_path, index=False)
    print(f"Batch {batch_number} saved to {file_path}")

# Main processing loop with tqdm for progress bar
def process_all_groups(df_hol, test_data_hol, all_groups, save_interval=100, save_path='predicted_arima_data_v2'):
    all_predicted_data = []
    batch_counter = 0  # To track number of processed batches

    # Loop through groups with tqdm
    for i, group in enumerate(tqdm(all_groups, desc="Processing Groups"), start=1):
        # Process group data
        res, x_test, end, start, predicted_df, train_data_f, test_data = Train_Predict(df_hol, test_data_hol, group)
        all_predicted_data.append(predicted_df)
        
        # Check if it's time to save the data
        if i % save_interval == 0:
            batch_counter += 1
            save_batch_data(all_predicted_data, save_path, batch_counter)
            all_predicted_data.clear()  # Clear the list after saving to free memory

    # Save remaining data if any after the loop ends
    if all_predicted_data:
        batch_counter += 1
        save_batch_data(all_predicted_data, save_path, batch_counter)

def process_all_groups_Prophet(df_hol, test_data_hol, all_groups, save_interval=100, save_path='predicted_prophet_data_v1'):
    all_predicted_data = []
    batch_counter = 0  # To track number of processed batches

    # Loop through groups with tqdm
    for i, group in enumerate(tqdm(all_groups, desc="Processing Groups"), start=1):
        # Process group data
        model,future_dfs,predicted_df,train_data_f,test_data = Train_Predict_Prophet(df_hol, test_data_hol, group)
        all_predicted_data.append(predicted_df)
        
        # Check if it's time to save the data
        if i % save_interval == 0:
            batch_counter += 1
            save_batch_data(all_predicted_data, save_path, batch_counter)
            all_predicted_data.clear()  # Clear the list after saving to free memory

    # Save remaining data if any after the loop ends
    if all_predicted_data:
        batch_counter += 1
        save_batch_data(all_predicted_data, save_path, batch_counter)

In [ ]:
count=0
all_df=pd.DataFrame()
for file in os.listdir('/kaggle/working/predicted_data'):
    count=count+1
    file_path=os.path.join('/kaggle/working/predicted_data',file)
    df=pd.read_csv(file_path)
    all_df=pd.concat([df,all_df],axis=0)
for file in os.listdir('/kaggle/working/predicted_data_v2'):
    count=count+1
    file_path=os.path.join('/kaggle/working/predicted_data_v2',file)
    df=pd.read_csv(file_path)
    all_df=pd.concat([df,all_df],axis=0)
count    

In [162]:
# all_groups = set(list(df_hol['default_rank'].unique())) - set(list(all_df['Group'].unique()))
# all_groups

In [ ]:
all_groups

In [164]:
df_hol.to_csv('df_hol.csv',index=False)
test_data_hol.to_csv('test_data_hol.csv',index=False)

In [212]:
from tqdm import tqdm
def call():
    if __name__ == '__main__':
          # Assuming 'group' is a column in df_hol
        process_all_groups_Prophet(df_hol, test_data_hol, all_groups, save_interval=10, save_path="predicted_prophet_data_v2")

In [ ]:
call()

In [ ]:
# model,future_dfs,predicted_df,train_data_f,test_data=Train_Predict_Prophet(df_hol,test_data_hol,77)

In [ ]:
# sns.lineplot(data=predicted_df,x=predicted_df.index,y=77)
# sns.lineplot(data=train_data_f,x='date',y='sales')

In [ ]:
# from multiprocessing import Pool, cpu_count

# # Define a wrapper function for multiprocessing
# def process_group(args):
#     df_hol, test_data_hol, group = args
#     res, x_test, end, start, predicted_df, train_data_f, test_data = Train_Predict(df_hol, test_data_hol, group)
#     return predicted_df

# # Prepare data for multiprocessing
# all_groups = df_hol['default_rank'].unique()  # Assuming 'group' is a column in df_hol
# args_list = [(df_hol, test_data_hol, group) for group in all_groups]  # Create argument tuples

# # Use multiprocessing to process groups in parallel
# if __name__ == '__main__':
#     with Pool(processes=cpu_count()) as pool:  # Use all available CPU cores
#         all_predicted_data = pool.map(process_group, args_list)


In [ ]:
# import pandas as pd
# from concurrent.futures import ThreadPoolExecutor, as_completed
# import os

# # Worker function to process each group
# def process_group(df_hol, test_data_hol, group):
#     try:
#         res, x_test, end, start, predicted_df, train_data_f, test_data = Train_Predict(df_hol, test_data_hol, group)
#         return predicted_df
#     except Exception as e:
#         print(f"Error processing group {group}: {e}")
#         return None

# # Function to run the process with multithreading
# def process_with_saving(df_hol, test_data_hol, all_groups, save_interval=300, save_path="predicted_data"):
#     if not os.path.exists(save_path):
#         os.makedirs(save_path)  # Create the directory if it doesn't exist

#     all_predicted_data = []  # List to store all predictions
#     futures = []
#     save_counter = 0  # Counter to track saving intervals

#     # Use ThreadPoolExecutor for multithreading
#     with ThreadPoolExecutor() as executor:
#         # Submit tasks for all groups
#         future_to_group = {
#             executor.submit(process_group, df_hol, test_data_hol, group): group for group in all_groups
#         }

#         # Collect results as they complete
#         for future in as_completed(future_to_group):
#             group = future_to_group[future]
#             try:
#                 result = future.result()
#                 if result is not None:
#                     all_predicted_data.append(result)
#             except Exception as e:
#                 print(f"Error processing group {group}: {e}")

#             # Save data at specified intervals
#             save_counter += 1
#             if save_counter % save_interval == 0:
#                 save_data(all_predicted_data, save_path, batch_number=save_counter // save_interval)
#                 all_predicted_data.clear()  # Clear the list after saving to free memory

#     # Save any remaining data after all groups are processed
#     if all_predicted_data:
#         save_data(all_predicted_data, save_path, batch_number=(save_counter // save_interval) + 1)

# # Function to save predicted data
# def save_data(data_list, save_path, batch_number):
#     # Combine all predicted data into a single DataFrame
#     combined_df = pd.concat(data_list, ignore_index=True)
#     file_name = f"predicted_batch_{batch_number}.csv"
#     file_path = os.path.join(save_path, file_name)
#     combined_df.to_csv(file_path, index=False)
#     print(f"Batch {batch_number} saved to {file_path}")


In [ ]:
# if __name__ == '__main__':
#     all_groups = df_hol['default_rank'].unique()  # Assuming 'group' is a column in df_hol
#     process_with_saving(df_hol, test_data_hol, all_groups, save_interval=300, save_path="predicted_data")


In [ ]:
print('hello')
import os
import pandas as pd

In [214]:
count=0
all_df=pd.DataFrame()
for file in os.listdir('/kaggle/working/predicted_prophet_data_v2'):
    count=count+1
    file_path=os.path.join('/kaggle/working/predicted_prophet_data_v2',file)
    df=pd.read_csv(file_path)
    all_df=pd.concat([df,all_df],axis=0)
    
    

In [ ]:
count

In [215]:
all_df.to_csv('allpreddata_round1_prophet_v3.csv',index=False)

In [ ]:
all_df[all_df.duplicated(subset='id')]

In [216]:
all_df[['id','yhat']].rename(columns={'yhat':'sales'}).to_csv('Prophet_Submission_v2.csv',index=False)

In [ ]:
all_df